In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import re
import streamlit as st

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\LAPTOP
[nltk_data]     INSIDE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\LAPTOP
[nltk_data]     INSIDE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\LAPTOP
[nltk_data]     INSIDE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.tag import pos_tag

In [5]:
from scipy.sparse import hstack

In [6]:
from sklearn.model_selection import GridSearchCV

In [27]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\LAPTOP
[nltk_data]     INSIDE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [39]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\LAPTOP INSIDE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [7]:
df_true = pd.read_csv('True.csv')
df_fake = pd.read_csv('Fake.csv')

In [8]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
print('Shape of True article file:',df_true.size)
print('Shape of Fake article file:',df_fake.size)

Shape of True article file: 85668
Shape of Fake article file: 93924


In [6]:
df_true.describe()

,title,text,subject,date
count,21417,21417,21417,21417
unique,20826,21192,2,716
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017"
freq,14,8,11272,182


In [7]:
df_fake.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [8]:
df_true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [9]:
df_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [20]:
df_true['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [12]:
df_true['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [21]:
df_fake['title'][0]

' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing'

In [13]:
df_fake['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [22]:
df_true['title'][1]

'U.S. military to accept transgender recruits on Monday: Pentagon'

In [14]:
df_true['text'][1]

'WASHINGTON (Reuters) - Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District Court in the meantime,” the official said, speaking on condition 

In [23]:
df_fake['title'][1]

' Drunk Bragging Trump Staffer Started Russian Collusion Investigation'

In [15]:
df_fake['text'][1]

'House Intelligence Committee Chairman Devin Nunes is going to have a bad day. He s been under the assumption, like many of us, that the Christopher Steele-dossier was what prompted the Russia investigation so he s been lashing out at the Department of Justice and the FBI in order to protect Trump. As it happens, the dossier is not what started the investigation, according to documents obtained by the New York Times.Former Trump campaign adviser George Papadopoulos was drunk in a wine bar when he revealed knowledge of Russian opposition research on Hillary Clinton.On top of that, Papadopoulos wasn t just a covfefe boy for Trump, as his administration has alleged. He had a much larger role, but none so damning as being a drunken fool in a wine bar. Coffee boys  don t help to arrange a New York meeting between Trump and President Abdel Fattah el-Sisi of Egypt two months before the election. It was known before that the former aide set up meetings with world leaders for Trump, but team Tr

In [10]:
df_true['label'] = 1
df_fake['label'] = 0

In [11]:
df_merged = pd.concat([df_true,df_fake],ignore_index=True)
df_merged = df_merged.sample(frac=1,random_state=42).reset_index(drop=True)
df_merged.head()

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0


In [12]:
df_merged['label'].value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

In [24]:
df_merged['subject'].value_counts()

subject
politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: count, dtype: int64

In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+|\b\w+\.com\b','',text)
    text = re.sub(r'@\w+','',text)
    text = re.sub(r"[^\w\s']",'',text)
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'\d+','',text)
    return text

In [14]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

In [15]:
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [16]:
df_merged['text_cleaned'] = df_merged['text'].apply(clean_text)
df_merged['title_cleaned'] = df_merged['title'].apply(clean_text)

df_merged['text_tokenized'] = df_merged['text_cleaned'].apply(word_tokenize)
df_merged['title_tokenized'] = df_merged['title_cleaned'].apply(word_tokenize)

df_merged['text_stopwords'] = df_merged['text_tokenized'].apply(remove_stopwords)
df_merged['title_stopwords'] = df_merged['title_tokenized'].apply(remove_stopwords)

df_merged['text_lemmatized'] = df_merged['text_stopwords'].apply(lemmatize_tokens)
df_merged['title_lemmatized'] = df_merged['title_stopwords'].apply(lemmatize_tokens)

df_merged.head()

,title,text,subject,date,label,text_cleaned,title_cleaned,text_tokenized,title_tokenized,text_stopwords,title_stopwords,text_lemmatized,title_lemmatized
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0,donald trump s white house is in chaos and the...,breaking gop chairman grassley has had enough...,"[donald, trump, s, white, house, is, in, chaos...","[breaking, gop, chairman, grassley, has, had, ...","[donald, trump, white, house, chaos, trying, c...","[breaking, gop, chairman, grassley, enough, de...","[donald, trump, white, house, chaos, trying, c...","[breaking, gop, chairman, grassley, enough, de..."
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0,now that donald trump is the presumptive gop n...,failed gop candidates remembered in hilarious...,"[now, that, donald, trump, is, the, presumptiv...","[failed, gop, candidates, remembered, in, hila...","[donald, trump, presumptive, gop, nominee, tim...","[failed, gop, candidates, remembered, hilariou...","[donald, trump, presumptive, gop, nominee, tim...","[failed, gop, candidate, remembered, hilarious..."
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0,mike pence is a huge homophobe he supports exg...,mike pences new dc neighbors are hilariously ...,"[mike, pence, is, a, huge, homophobe, he, supp...","[mike, pences, new, dc, neighbors, are, hilari...","[mike, pence, huge, homophobe, supports, exgay...","[mike, pences, new, dc, neighbors, hilariously...","[mike, penny, huge, homophobe, support, exgay,...","[mike, pences, new, dc, neighbor, hilariously,..."
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1,san francisco reuters california attorney gene...,california ag pledges to defend birth control ...,"[san, francisco, reuters, california, attorney...","[california, ag, pledges, to, defend, birth, c...","[san, francisco, reuters, california, attorney...","[california, ag, pledges, defend, birth, contr...","[san, francisco, reuters, california, attorney...","[california, ag, pledge, defend, birth, contro..."
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0,twisted reasoning is all that comes from pelos...,az ranchers living on usmexico border destroy ...,"[twisted, reasoning, is, all, that, comes, fro...","[az, ranchers, living, on, usmexico, border, d...","[twisted, reasoning, comes, pelosi, days, espe...","[az, ranchers, living, usmexico, border, destr...","[twisted, reasoning, come, pelosi, day, especi...","[az, rancher, living, usmexico, border, destro..."


In [17]:
df_merged[['title','title_cleaned','title_lemmatized','text','text_cleaned','text_lemmatized']].head()

,title,title_cleaned,title_lemmatized,text,text_cleaned,text_lemmatized
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,breaking gop chairman grassley has had enough...,"[breaking, gop, chairman, grassley, enough, de...","Donald Trump s White House is in chaos, and th...",donald trump s white house is in chaos and the...,"[donald, trump, white, house, chaos, trying, c..."
1,Failed GOP Candidates Remembered In Hilarious...,failed gop candidates remembered in hilarious...,"[failed, gop, candidate, remembered, hilarious...",Now that Donald Trump is the presumptive GOP n...,now that donald trump is the presumptive gop n...,"[donald, trump, presumptive, gop, nominee, tim..."
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,mike pences new dc neighbors are hilariously ...,"[mike, pences, new, dc, neighbor, hilariously,...",Mike Pence is a huge homophobe. He supports ex...,mike pence is a huge homophobe he supports exg...,"[mike, penny, huge, homophobe, support, exgay,..."
3,California AG pledges to defend birth control ...,california ag pledges to defend birth control ...,"[california, ag, pledge, defend, birth, contro...",SAN FRANCISCO (Reuters) - California Attorney ...,san francisco reuters california attorney gene...,"[san, francisco, reuters, california, attorney..."
4,AZ RANCHERS Living On US-Mexico Border Destroy...,az ranchers living on usmexico border destroy ...,"[az, rancher, living, usmexico, border, destro...",Twisted reasoning is all that comes from Pelos...,twisted reasoning is all that comes from pelos...,"[twisted, reasoning, come, pelosi, day, especi..."


In [18]:
def lemmatize_tokens_pos(tokens):
    lemmatized_tokens = []
    pos_tags = pos_tag(tokens)
    for token, tag in pos_tags:
        if tag.startswith('VB'):
            lemma = lemmatizer.lemmatize(token, pos='v')
        else:
            lemma = lemmatizer.lemmatize(token)
        lemmatized_tokens.append(lemma)
    return lemmatized_tokens

In [20]:
df_merged['text_lemmatized_pos'] = df_merged['text_stopwords'].apply(lemmatize_tokens_pos)
df_merged['title_lemmatized_pos'] = df_merged['title_stopwords'].apply(lemmatize_tokens_pos)

In [21]:
df_merged['text_lemmatized'][0]

['donald',
 'trump',
 'white',
 'house',
 'chaos',
 'trying',
 'cover',
 'russia',
 'problem',
 'mounting',
 'hour',
 'refuse',
 'acknowledge',
 'problem',
 'surrounding',
 'fake',
 'news',
 'hoax',
 'however',
 'fact',
 'bear',
 'thing',
 'differently',
 'seems',
 'crack',
 'congressional',
 'public',
 'leadershipchuck',
 'grassley',
 'riowa',
 'head',
 'senate',
 'judiciary',
 'committee',
 'fed',
 'demanding',
 'donald',
 'trump',
 'jr',
 'former',
 'trump',
 'campaign',
 'manager',
 'paul',
 'manafort',
 'testify',
 'committee',
 'regarding',
 'infamous',
 'shady',
 'meeting',
 'donald',
 'trump',
 'shady',
 'russian',
 'lawyer',
 'promised',
 'dirt',
 'democratic',
 'presidential',
 'nominee',
 'hillary',
 'clinton',
 'fact',
 'information',
 'due',
 'well',
 'demand',
 'sends',
 'signal',
 'team',
 'trump',
 'notably',
 'fire',
 'special',
 'counsel',
 'robert',
 'mueller',
 'circumstance',
 'despite',
 'fact',
 'seems',
 'seems',
 'trump',
 'white',
 'house',
 'laying',
 'ground

In [22]:
df_merged['text_lemmatized_pos'][0]

['donald',
 'trump',
 'white',
 'house',
 'chaos',
 'try',
 'cover',
 'russia',
 'problem',
 'mount',
 'hour',
 'refuse',
 'acknowledge',
 'problem',
 'surround',
 'fake',
 'news',
 'hoax',
 'however',
 'facts',
 'bear',
 'thing',
 'differently',
 'seem',
 'crack',
 'congressional',
 'public',
 'leadershipchuck',
 'grassley',
 'riowa',
 'head',
 'senate',
 'judiciary',
 'committee',
 'feed',
 'demand',
 'donald',
 'trump',
 'jr',
 'former',
 'trump',
 'campaign',
 'manager',
 'paul',
 'manafort',
 'testify',
 'committee',
 'regard',
 'infamous',
 'shady',
 'meeting',
 'donald',
 'trump',
 'shady',
 'russian',
 'lawyer',
 'promise',
 'dirt',
 'democratic',
 'presidential',
 'nominee',
 'hillary',
 'clinton',
 'fact',
 'information',
 'due',
 'well',
 'demand',
 'send',
 'signal',
 'team',
 'trump',
 'notably',
 'fire',
 'special',
 'counsel',
 'robert',
 'mueller',
 'circumstance',
 'despite',
 'fact',
 'seem',
 'seem',
 'trump',
 'white',
 'house',
 'lay',
 'groundwork',
 'speak',
 'sp

In [23]:
def join_tokens(token):
    return " ".join(token)

In [24]:
df_merged['title_final'] = df_merged['title_lemmatized_pos'].apply(join_tokens)
df_merged['text_final'] = df_merged['text_lemmatized_pos'].apply(join_tokens)

print(df_merged[['title_lemmatized_pos','title_final']].head())
print(df_merged[['text_lemmatized_pos','text_final']].head())

                                title_lemmatized_pos  \
0  [break, gop, chairman, grassley, enough, deman...   
1  [failed, gop, candidate, remember, hilarious, ...   
2  [mike, pences, new, dc, neighbor, hilariously,...   
3  [california, ag, pledge, defend, birth, contro...   
4  [az, rancher, live, usmexico, border, destroy,...   

                                         title_final  
0  break gop chairman grassley enough demand trum...  
1  failed gop candidate remember hilarious mock e...  
2  mike pences new dc neighbor hilariously troll ...  
3  california ag pledge defend birth control insu...  
4  az rancher live usmexico border destroy nancy ...  
                                 text_lemmatized_pos  \
0  [donald, trump, white, house, chaos, try, cove...   
1  [donald, trump, presumptive, gop, nominee, tim...   
2  [mike, penny, huge, homophobe, support, exgay,...   
3  [san, francisco, reuters, california, attorney...   
4  [twisted, reasoning, come, pelosi, day, especi... 

In [25]:
df_merged.shape

(44898, 17)

In [26]:
X_test = df_merged['text_final']
X_title = df_merged['title_final']
y = df_merged['label']

X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(X_test,y,test_size=0.2,random_state=42)
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(X_title,y,test_size=0.2,random_state=42)

y_train, y_test = y_train_text, y_test_text 

In [27]:
print("Shape of X_train_text:", X_train_text.shape)
print("Shape of X_test_text:", X_test_text.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train_text: (35918,)
Shape of X_test_text: (8980,)
Shape of y_train: (35918,)
Shape of y_test: (8980,)


In [28]:
tfidf_vetorizer_text = TfidfVectorizer(max_features=5000)
tfidf_vetorizer_title = TfidfVectorizer(max_features=5000)

In [29]:
tfidf_matrix_train_text = tfidf_vetorizer_text.fit_transform(X_train_text)
tfidf_matrix_test_text = tfidf_vetorizer_text.transform(X_test_text)

tfidf_matrix_train_title = tfidf_vetorizer_title.fit_transform(X_train_title)
tfidf_matrix_test_title = tfidf_vetorizer_title.transform(X_test_title)

In [30]:
print("Shape of text TF-IDF matrix (training):", tfidf_matrix_train_text.shape)
print("Shape of text TF-IDF matrix (testing):", tfidf_matrix_test_text.shape)
print("Shape of title TF-IDF matrix (training):", tfidf_matrix_train_title.shape)
print("Shape of title TF-IDF matrix (testing):", tfidf_matrix_test_title.shape)

Shape of text TF-IDF matrix (training): (35918, 5000)
Shape of text TF-IDF matrix (testing): (8980, 5000)
Shape of title TF-IDF matrix (training): (35918, 5000)
Shape of title TF-IDF matrix (testing): (8980, 5000)


In [31]:
tfidf_train_combined = hstack([tfidf_matrix_train_title, tfidf_matrix_train_text])
tfidf_test_combined = hstack([tfidf_matrix_test_title, tfidf_matrix_test_text])

In [32]:
naive_bayes_classifier = MultinomialNB()
random_forest_classifier = RandomForestClassifier(random_state=42)

In [33]:
naive_bayes_classifier.fit(tfidf_train_combined,y_train)
random_forest_classifier.fit(tfidf_train_combined,y_train)

RandomForestClassifier(random_state=42)

In [34]:
naive_bayes_pred = naive_bayes_classifier.predict(tfidf_test_combined)
random_forest_pred = random_forest_classifier.predict(tfidf_test_combined)

In [35]:
print("Multinomial Naive Bayes Classifier (Combined Text and Title Features):")
print("Accuracy:", accuracy_score(y_test, naive_bayes_pred))
print("Classification Report:\n", classification_report(y_test, naive_bayes_pred))

print("\nRandom Forest Classifier (Combined Text and Title Features):")
print("Accuracy:", accuracy_score(y_test, random_forest_pred))
print("Classification Report:\n", classification_report(y_test, random_forest_pred))

Multinomial Naive Bayes Classifier (Combined Text and Title Features):
Accuracy: 0.9459910913140311
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95      4669
           1       0.95      0.94      0.94      4311

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980


Random Forest Classifier (Combined Text and Title Features):
Accuracy: 0.9969933184855234
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4669
           1       1.00      1.00      1.00      4311

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [36]:
naive_bayes_classifier.fit(tfidf_matrix_train_text,y_train)
random_forest_classifier.fit(tfidf_matrix_train_text,y_train)

RandomForestClassifier(random_state=42)

In [37]:
nb_pred_text = naive_bayes_classifier.predict(tfidf_matrix_test_text)
rf_pred_text = random_forest_classifier.predict(tfidf_matrix_test_text)

In [38]:
print("Multinomial Naive Bayes Classifier for text:")
print("Accuracy:", accuracy_score(y_test, nb_pred_text))
print("Classification Report:\n", classification_report(y_test, nb_pred_text))

print("\nRandom Forest Classifier for text:")
print("Accuracy:", accuracy_score(y_test, rf_pred_text))
print("Classification Report:\n", classification_report(y_test, rf_pred_text))

Multinomial Naive Bayes Classifier for text:
Accuracy: 0.9280623608017817
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93      4669
           1       0.93      0.92      0.92      4311

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980


Random Forest Classifier for text:
Accuracy: 0.9973273942093541
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4669
           1       1.00      1.00      1.00      4311

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [83]:
param_grid_rf = {'n_estimators':[100,200,300],
                'max_depth':[10,20,30,None],
                'min_samples_split':[2,5,10],
                'min_samples_leaf':[1,2,4]}

In [84]:
grid_search_cv = GridSearchCV(estimator=random_forest_classifier,param_grid=param_grid_rf,cv=3,scoring='accuracy',n_jobs=1)

In [85]:
grid_search_cv.fit(tfidf_train_combined,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=1,
             param_grid={'max_depth': [10, 20, 30, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [86]:
best_rf_classifier = grid_search_cv.best_estimator_

In [87]:
best_rf_classifier

RandomForestClassifier(min_samples_split=5, n_estimators=200, random_state=42)

In [88]:
best_rf_pred = best_rf_classifier.predict(tfidf_test_combined)

In [89]:
print("Best Random Forest Classifier (after Hyperparameter Tuning with GridSearchCV):")
print("Best Hyperparameters:", grid_search_cv.best_params_) # Print the best hyperparameters found
print("Accuracy on Test Set:", accuracy_score(y_test, best_rf_pred))
print("Classification Report on Test Set:\n", classification_report(y_test, best_rf_pred))

Best Random Forest Classifier (after Hyperparameter Tuning with GridSearchCV):
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy on Test Set: 0.9976614699331848
Classification Report on Test Set:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4669
           1       1.00      1.00      1.00      4311

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [90]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [92]:
tokenizer_text_lstm = Tokenizer(num_words=5000,oov_token='<oov>')
tokenizer_title_lstm = Tokenizer(num_words=5000,oov_token='<oov>')

In [94]:
tokenizer_text_lstm.fit_on_texts(X_train_text)

In [101]:
word_index_text = tokenizer_text_lstm.word_index
print('Vocabulary size for text:',len(word_index_text))

Vocabulary size for text: 170540


In [102]:
sequence_train_text = tokenizer_text_lstm.texts_to_sequences(X_train_text)
padded_train_text = pad_sequences(sequence_train_text,padding='post',maxlen=200)

In [103]:
sequence_test_text = tokenizer_text_lstm.texts_to_sequences(X_test_text)
padded_test_text = pad_sequences(sequence_test_text,padding='post',maxlen=200)

In [104]:
tokenizer_title_lstm.fit_on_texts(X_train_title)

In [105]:
word_index_title = tokenizer_title_lstm.word_index
print('Vocabulary size of title:',len(word_index_title))

Vocabulary size of title: 20044


In [110]:
sequence_train_title = tokenizer_title_lstm.texts_to_sequences(X_train_title)
padded_train_title = pad_sequences(sequence_train_title,padding='post',maxlen=20)

In [111]:
sequence_test_title = tokenizer_title_lstm.texts_to_sequences(X_test_title)
padded_test_title = pad_sequences(sequence_test_title,padding='post',maxlen=20)

In [108]:
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

In [112]:
print("\nPadded training text data shape:", padded_train_text.shape)
print("Padded testing text data shape:", padded_test_text.shape)
print("Padded training title data shape:", padded_train_title.shape)
print("Padded testing title data shape:", padded_test_title.shape)
print("Training labels shape:", y_train_np.shape)
print("Testing labels shape:", y_test_np.shape)


Padded training text data shape: (35918, 200)
Padded testing text data shape: (8980, 200)
Padded training title data shape: (35918, 20)
Padded testing title data shape: (8980, 20)
Training labels shape: (35918,)
Testing labels shape: (8980,)


In [113]:
from keras.models import Model
from keras.layers import Input, Concatenate, Dropout

In [117]:
input_text = Input(shape=(200,))
input_title = Input(shape=(20,))

embedding_dim = 128
embedding_text = Embedding(input_dim=5000,
                          output_dim=embedding_dim,
                          input_length=200)(input_text)

embedding_title = Embedding(input_dim=5000,
                           output_dim=embedding_dim,
                           input_length=20)(input_title)

lstm_text = LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)(embedding_text)
lstm_title = LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)(embedding_title)

concatenated_layer = Concatenate()
concatenated_output = concatenated_layer([lstm_title,lstm_text])

dense1 = Dense(128, activation='relu')(concatenated_output)
dropout1 = Dropout(0.5)(dense1)

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)

output = Dense(1, activation='sigmoid')(dropout2)

lstm_model = Model(inputs=[input_title,input_text],outputs=output, name='fake_news_lstm_model')
lstm_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

lstm_model.summary()

Model: "fake_news_lstm_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 200)]                0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 20, 128)              640000    ['input_4[0][0]']             
                                                                                                  
 embedding_2 (Embedding)     (None, 200, 128)             640000    ['input_3[0][0]']             
                                                                               

In [118]:
history_lstm = lstm_model.fit(x=[padded_train_title,padded_train_text], y=y_train_np, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
225/225 [==============================] - 485s 2s/step - loss: 0.1808 - accuracy: 0.9169 - val_loss: 0.0851 - val_accuracy: 0.9716
Epoch 2/10
225/225 [==============================] - 660s 3s/step - loss: 0.0369 - accuracy: 0.9897 - val_loss: 0.0401 - val_accuracy: 0.9857
Epoch 3/10
225/225 [==============================] - 668s 3s/step - loss: 0.0151 - accuracy: 0.9959 - val_loss: 0.0443 - val_accuracy: 0.9859
Epoch 4/10
225/225 [==============================] - 676s 3s/step - loss: 0.0198 - accuracy: 0.9938 - val_loss: 0.0927 - val_accuracy: 0.9699
Epoch 5/10
225/225 [==============================] - 674s 3s/step - loss: 0.0459 - accuracy: 0.9853 - val_loss: 0.0740 - val_accuracy: 0.9755
Epoch 6/10
225/225 [==============================] - 672s 3s/step - loss: 0.0227 - accuracy: 0.9926 - val_loss: 0.0726 - val_accuracy: 0.9788
Epoch 7/10
225/225 [==============================] - 681s 3s/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 0.1000 - val_accuracy: 0.9809

In [120]:
loss_lstm, accuracy_lstm = lstm_model.evaluate(x=[padded_test_title,padded_test_text],y=y_test_np)

281/281 [==============================] - 16s 57ms/step - loss: 0.0867 - accuracy: 0.9836


In [121]:
print(f"\nLSTM Model - Test Set Evaluation:")
print(f"Test Loss: {loss_lstm:.4f}")
print(f"Test Accuracy: {accuracy_lstm:.4f}")


LSTM Model - Test Set Evaluation:
Test Loss: 0.0867
Test Accuracy: 0.9836


In [122]:
lstm_pred_probs = lstm_model.predict([padded_test_title,padded_test_text])
lstm_pred_binary = (lstm_pred_probs > 0.5).astype(int)
lstm_pred_flat = lstm_pred_binary.flatten()

281/281 [==============================] - 17s 58ms/step


In [124]:
print("\nLSTM Model - Classification Report on Test Set:")
print(classification_report(y_test_np, lstm_pred_flat))


LSTM Model - Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4669
           1       0.98      0.98      0.98      4311

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [125]:
print("\nLSTM Model - Confusion Matrix on Test Set:")
print(confusion_matrix(y_test_np, lstm_pred_flat))


LSTM Model - Confusion Matrix on Test Set:
[[4589   80]
 [  67 4244]]


In [126]:
import joblib

In [127]:
model_filename = 'fake_news_rf_model.joblib'
joblib.dump(best_rf_classifier,model_filename)
print(f'Trained Random Forest Model saved to: {model_filename}')

Trained Random Forest Model saved to: fake_news_rf_model.joblib


In [128]:
vectorizer_text_filename = 'tfidf_vectorizer_text.joblib'
vectorizer_title_filename = 'tfidf_vectorizer_title.joblib'

joblib.dump(tfidf_vetorizer_text,vectorizer_text_filename)
joblib.dump(tfidf_vetorizer_title,vectorizer_title_filename)

print(f"TF-IDF vectorizer (text) saved to: {vectorizer_text_filename}")
print(f"TF-IDF vectorizer (title) saved to: {vectorizer_title_filename}")

TF-IDF vectorizer (text) saved to: tfidf_vectorizer_text.joblib
TF-IDF vectorizer (title) saved to: tfidf_vectorizer_title.joblib


In [1]:
def predict_fake_news(title_content, text_content):
    loaded_vectorizer_title = joblib.load('tfidf_vectorizer_title.joblib')
    loaded_vectorizer_text = joblib.load('tfidf_vectorizer_text.joblib')
    loaded_rf_model = joblib.load('fake_news_rf_model.joblib')

    title_features = loaded_vectorizer_title.transform([title_content])
    text_features = loaded_vectorizer_text.transform([text_content])

    combined_features = np.hstack([title_features.toarray(), text_features.toarray()])

    prediction_rf = loaded_rf_model.predict(combined_features)

    if prediction_rf[0] == 1:
        prediction_label = 'Fake news'
    else:
        prediction_label = 'Real news'
        
    return prediction_label
    